Load several functions:

    * get_site_name_from_park_boundary()
    
    * apply_site_name_query(site_name_value,
                            aprx = arcpy.mp.ArcGISProject("CURRENT"),
                            map_name = None)
                            
    * remove_queries(aprx = arcpy.mp.ArcGISProject("CURRENT"),
                     map_name = None)
                     
    * zoom_to_active_park(aprx = arcpy.mp.ArcGISProject("CURRENT"))
    
    * update_layer_datasources(aprx = arcpy.mp.ArcGISProject("CURRENT"),
                               map_name = None, 
                               old_path = r"C:\Users\Spencer_C\Desktop\GDB Copy\FloridaStateParksGIS.gdb", 
                               new_path = r"\\floridadep\data\DRP\GIS\statewidecoverages\FloridaStateParksGIS.gdb")
                               
    * save_layout_file(output_folder=r'C:\Users\Spencer_C\Documents\UMP_Automation_Testing\testing', 
                       aprx=arcpy.mp.ArcGISProject("CURRENT"), 
                       layout_name=None, 
                       pagx_filename=None)
    

### Get Site Name
Getting the site name is pretty useless on its own, but can be useful when combined with other functions. Return the selected site name. If no selection is made, return all the 'site name' values in the layer - with a dqs applied this would only return the dq values. 

In [55]:
def get_site_name_from_park_boundary():
    """
    Returns the 'site_name' attribute(s) of selected features in the
    'Park Boundary' layer of the current map in ArcGIS Pro.

    Returns:
        list: A list of 'site_name' values from the selected features. If no features are selected, returns all site name values in the table
    """
    site_names = []

    # Access the current ArcGIS Pro project and the active map
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    active_map = aprx.activeMap

    # Attempt to get the 'Park Boundary' layer
    park_layer = None
    for lyr in active_map.listLayers():
        if lyr.name.lower() == "park boundary" and not lyr.isGroupLayer:
            park_layer = lyr
            break

    if not park_layer:
        print("Layer 'Park Boundary' not found or not valid.")
        return site_names

    # Check for 'site_name' field
    fields = [f.name.lower() for f in arcpy.ListFields(park_layer)]
    if "site_name" not in fields:
        print("Field 'site_name' not found in 'Park Boundary' layer.")
        return site_names

    # Get selected feature IDs
    selected_oids = park_layer.getSelectionSet()
    
    # If no selection made, return all the park names in the layer
    if not selected_oids:
        print("No features selected in 'Park Boundary' layer.")
        with arcpy.da.SearchCursor(park_layer, ["site_name"]) as cursor:
            for row in cursor:
                site_names.append(row[0])

    # Use SearchCursor to fetch 'site_name' values
    with arcpy.da.SearchCursor(park_layer, ["site_name"], where_clause=f"OBJECTID IN ({','.join(map(str, selected_oids))})") as cursor:
        for row in cursor:
            site_names.append(row[0])

    return site_names

In [63]:
get_site_name_from_park_boundary()

['Lake Jackson Mounds Archaeological State Park', 'Tallahassee-St. Marks Historic Railroad State Trail', 'Alfred B. Maclay Gardens State Park', 'Edward Ball Wakulla Springs State Park']

Apply site name queries to all layers in a map. The site name query can contain multiple values or just one.

In [60]:
def apply_site_name_query(site_name_value,
                          aprx = arcpy.mp.ArcGISProject("CURRENT"),
                          map_name = None):
    """
    Applies a definition query to all layers in the given map object
    where the layer has a 'site_name' field. The query filters the layer
    to only include features where site_name equals the input string.

    Parameters:
        map_name (str): The map name containing layers.
        site_name_value (str): The site name value to filter by.
        aprx (project object): The project to look for the map in. The default value is the current project.
    """
    # Get name of active map
    if not map_name:
        map_name = aprx.activeMap.name
        
    # Get map object using the map name
    try:
        map_obj = aprx.listMaps(map_name)[0]
    except Exception as e:
        print(f"Could not find {map_name}: {e}")
    
    # Get DQ values from site_name_value
    if isinstance(site_name_value, str):
        site_name_value = [site_name_value]
    dq_vals = '\',\''.join(site_name_value)
    
    # Apply definition queries to all appropriate layers in the map       
    for lyr in map_obj.listLayers():
        if lyr.supports("DEFINITIONQUERY") and not lyr.isGroupLayer:
            try:
                # Check if 'site_name' field exists
                desc = arcpy.Describe(lyr)
                fields = [f.name.lower() for f in arcpy.ListFields(lyr)]
                if 'site_name' in fields:
                    # Apply definition query
                    dql = lyr.listDefinitionQueries()
                    for dq in dql:
                        dq['isActive'] = False
                    #lyr.updateDefinitionQueries([{'name': 'Site Name Query', 'sql': f"site_name IN {site_name_value}", 'isActive': True}])
                    lyr.updateDefinitionQueries([{'name': 'Site Name Query', 'sql': f"site_name IN (\'{dq_vals}\')", 'isActive': True}])
                    #print(f"Applied query to {lyr.name}: {query}")
            except Exception as e:
                print(f"Could not apply query to {lyr.name}: {e}")

In [61]:
apply_site_name_query(get_site_name_from_park_boundary())

In [15]:
apply_site_name_query('John Gorrie Museum State Park')

In [3]:
def remove_queries(aprx = arcpy.mp.ArcGISProject("CURRENT"),
                   map_name = None):
    """
    Remove all definition queries added by apply_site_name_query().

    Parameters:
        map_name (str): The map name containing layers.
        aprx (project object): The project to look for the map in. The default value is the current project.
    """
    # Get name of active map
    if not map_name:
        map_name = aprx.activeMap.name
    # Get map object from name
    try:
        map_obj = aprx.listMaps(map_name)[0]
    except Exception as e:
        print(f"Could not find {map_name}: {e}")
    # Apply definition queries to all appropriate layers in the map    
    for lyr in map_obj.listLayers():
        if lyr.supports("DEFINITIONQUERY") and lyr.supports("DATASOURCE") and not lyr.isGroupLayer:
            try:
                # Check if 'site_name' field exists
                desc = arcpy.Describe(lyr)
                fields = [f.name.lower() for f in arcpy.ListFields(lyr)]
                if 'site_name' in fields:
                    # Apply definition query
                    dql = lyr.listDefinitionQueries()
                    for dq in dql:
                        dq['isActive'] = False
                    lyr.updateDefinitionQueries([{}])
                    #print(f"Applied query to {lyr.name}: {query}")
            except Exception as e:
                print(f"Could not apply query to {lyr.name}: {e}")

In [93]:
remove_queries()

In [16]:
def zoom_to_active_park(aprx = arcpy.mp.ArcGISProject("CURRENT"),
                        layer = 'Park Boundary'):
    """
    Zooms to the park that is selected in the active map.

    Parameters:
        aprx (project): the project to change the active view of.
        layer (str): the layer to zoom to. Default value is 'Park Boundary'.
    """
    mv = aprx.activeView # mapview
    m = mv.map # map
    try:
        mv.camera.setExtent(mv.getLayerExtent(m.listLayers(layer)[0]))
    except Exception as e:
        print(f"Could not zoom to the Park Boundry layer: {e}")

In [62]:
zoom_to_active_park()

it would be nice to be able to autimatically set reference scale. Reference scale in arcpy is the integer value of the scale ratio 1:x. If the refernce scale is not set, it's value is 0.

In [64]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.activeMap
print(m.referenceScale)

755.0601623502109


In [ ]:
'''
import os

def update_layer_datasources(aprx = arcpy.mp.ArcGISProject("CURRENT"),
                             map_name = None, 
                             old_path = r"C:\Users\Spencer_C\Desktop\GDB Copy\FloridaStateParksGIS.gdb", 
                             new_path = r"\\floridadep\data\DRP\GIS\statewidecoverages\FloridaStateParksGIS.gdb"):
    """
    Replaces the workspace path of each layer in the map from the local GDB
    to the network GDB.

    Parameters:
        aprx (project): project that contains the maps and the layers. The default argument is the current project.
        map_name (str): The name of the map containing layers to update. The default argument is the 
        old_path (str): the file path to the geodatabase to convert from.
        new_path (str): the file path to the geodatabase to convert to
    """
    # Get name of active map
    if not map_name:
        map_name = aprx.activeMap.name
        
    # Get map object from name
    try:
        map_obj = aprx.listMaps(map_name)[0]
    except Exception as e:
        print(f"Could not find {map_name}: {e}")
    
    # Update layer datasources
    for lyr in map_obj.listLayers():
        if lyr.supports("DATASOURCE") and not lyr.isGroupLayer:
            try:
                # Check if the layer is using the old path
                current_path = lyr.dataSource
                if old_path.lower() in current_path.lower():
                    # Extract dataset name (feature class name)
                    dataset_name = os.path.basename(current_path)
                    # Update the data source
                    lyr.updateConnectionProperties(old_path, new_path, validate=True)
                    #print(f"Updated {lyr.name} to use data from: {new_path}\\{dataset_name}")
            except Exception as e:
                print(f"Failed to update {lyr.name}: {e}")
'''

EDIT DATA SOURCES IN LAYOUT FILE
Editing data sources before a layout file is created is slow and prevents moving to a new task. Changing the data sources of a layour file that is not open

this script works

In [10]:
'''
#import os
old_path = r"C:\Users\Spencer_C\Desktop\GDB Copy\FloridaStateParksGIS.gdb" 
new_path = r"\\floridadep\data\DRP\GIS\statewidecoverages\FloridaStateParksGIS.gdb"
layout = arcpy.mp.ConvertLayoutFileToLayout(r'C:\Users\Spencer_C\Documents\UMP_Automation_Testing\testing\testing.pagx')

for mf in layout.listElements('MAPFRAME_ELEMENT'):
    map_obj = mf.map
    if map_obj:
        for lyr in map_obj.listLayers():
            if lyr.supports("DATASOURCE") and not lyr.isGroupLayer:
                try:
                    # Check if the layer is using the old path
                    current_path = lyr.dataSource
                    if old_path.lower() in current_path.lower():
                        # Extract dataset name (feature class name)
                        #dataset_name = os.path.basename(current_path)
                        # Update the data source
                        lyr.updateConnectionProperties(old_path, new_path, validate=True)
                        print(f"Updated {lyr.name} to use data from: {new_path}")
                except Exception as e:
                    print(f"Failed to update {lyr.name}: {e}")

layout.exportToPAGX(r'C:\Users\Spencer_C\Documents\UMP_Automation_Testing\testing\Base Map 8.5x11P_updatedSources4.pagx')
'''

In [26]:
# example path: r'C:\Users\Spencer_C\Documents\UMP_Automation_Testing\testing\testing.pagx'
def update_layout_datasources(pagx_path):
    old_path = r"C:\Users\Spencer_C\Desktop\GDB Copy\FloridaStateParksGIS.gdb" 
    new_path = r"\\floridadep\data\DRP\GIS\statewidecoverages\FloridaStateParksGIS.gdb"
    layout = arcpy.mp.ConvertLayoutFileToLayout(pagx_path)

    for mf in layout.listElements('MAPFRAME_ELEMENT'):
        map_obj = mf.map
        if map_obj:
            for lyr in map_obj.listLayers():
                if lyr.supports("DATASOURCE") and not lyr.isGroupLayer:
                    try:
                        # Check if the layer is using the old path
                        current_path = lyr.dataSource
                        if old_path.lower() in current_path.lower():
                            # Extract dataset name (feature class name)
                            # Update the data source
                            lyr.updateConnectionProperties(old_path, new_path, validate=True)
                            print(f"Updated {lyr.name} to use data from: {new_path}")
                    except Exception as e:
                        print(f"Failed to update {lyr.name}: {e}")

    layout.exportToPAGX(pagx_path)

In [44]:
update_layout_datasources()

Failed to update Campsites: name 'os' is not defined
Failed to update Special Use Points: name 'os' is not defined
Failed to update Trails: name 'os' is not defined
Failed to update Walkways: name 'os' is not defined
Failed to update Park Roads: name 'os' is not defined
Failed to update Special Use Areas: name 'os' is not defined
Failed to update Structures: name 'os' is not defined
Failed to update Marine Structures: name 'os' is not defined
Failed to update Parking Lots: name 'os' is not defined
Failed to update Park Boundary: name 'os' is not defined


In [ ]:
'''
def update_layout_datasource(output_folder=r'C:\Users\Spencer_C\Documents\UMP_Automation_Testing\testing', 
                                  aprx=arcpy.mp.ArcGISProject("CURRENT"), 
                                  layout_name=None, 
                                  pagx_filename=None,
                                  old_path = r"C:\Users\Spencer_C\Desktop\GDB Copy\FloridaStateParksGIS.gdb", 
                                  new_path = r"\\floridadep\data\DRP\GIS\statewidecoverages\FloridaStateParksGIS.gdb"):
    
    # Determine filename
    if not pagx_filename:
        pagx_filename = f"{layout.name}.pagx"
    pagx_path = os.path.join(output_folder, pagx_filename)
    
    # change data source
    layout = arcpy.mp.ConvertLayoutFileToLayout(pagx_path)
    
    for mf in layout.listElements('MAPFRAME_ELEMENT'):
        map_obj = mf.map
        if map_obj:
            for lyr in map_obj.listLayers():
                if lyr.supports("DATASOURCE") and not lyr.isGroupLayer:
                    try:
                        # Check if the layer is using the old path
                        current_path = lyr.dataSource
                        if old_path.lower() in current_path.lower():
                            # Extract dataset name (feature class name)
                            dataset_name = os.path.basename(current_path)
                            # Update the data source
                            lyr.updateConnectionProperties(old_path, new_path, validate=True)
                            print(f"Updated {lyr.name} to use data from: {new_path}\\{dataset_name}")
                    except Exception as e:
                        print(f"Failed to update {lyr.name}: {e}")

    layout.exportToPAGX(pagx_filename)
'''

In [22]:
## DOESNT WORK ##
'''
def update_layout_datasources(pagx_path = r'C:\Users\Spencer_C\Documents\UMP_Automation_Testing\testing\testing.pagx',
                              old_path = r"C:\Users\Spencer_C\Desktop\GDB Copy\FloridaStateParksGIS.gdb", 
                              new_path = r"\\floridadep\data\DRP\GIS\statewidecoverages\FloridaStateParksGIS.gdb"):
    """
    Loads a .pagx layout file, updates layer data sources from old_gdb to new_gdb.

    Parameters:
    - pagx_path: Full path to the .pagx file.
    - old_gdb: The old geodatabase filename to be replaced (default: 'old.gdb').
    - new_gdb: The new geodatabase filename to replace with (default: 'new.gdb').
    """
    # Load the layout from the .pagx file
    layout = arcpy.mp.ConvertLayoutFileToLayout(pagx_path)

    print(f"Processing layout: {pagx_path}")

    # Get all map frames in the layout
    map_frames = layout.listElements("MAPFRAME_ELEMENT")
    for map_frame in map_frames:
        map_obj = map_frame.map
        if map_obj:
            print(f"  MapFrame: {map_frame.name} | Map: {map_obj.name}")

            for lyr in map_obj.listLayers():
                if lyr.supports("DATASOURCE"):
                    datasource = lyr.dataSource
                    if old_path in datasource:
                        new_datasource = datasource.replace(old_path, new_path)
                        try:
                            lyr.updateConnectionProperties(datasource, new_datasource, validate = True)
                            print(f"    Updated: {lyr.name}")
                            print(f"      Old: {datasource}")
                            print(f"      New: {new_datasource}")
                        except Exception as e:
                            print(f"    Failed to update {lyr.name}: {e}")
                    else:
                        print(f"    Skipped (no match): {lyr.name}")
                else:
                    print(f"    Skipped (no datasource): {lyr.name}")
    layout.exportToPAGX(pagx_path)
'''

In [ ]:
update_layout_datasources()

In [ ]:
save_layout_file(layout_name = 'Base Map 8.5x11P', pagx_filename = 'testing')

In [ ]:
update_layout_datasource(layout_name = 'Base Map 8.5x11P', pagx_filename = 'testing')

In [21]:
def save_layout_file(output_folder=r'C:\Users\Spencer_C\Documents\UMP_Automation_Testing\testing', 
                     aprx=arcpy.mp.ArcGISProject("CURRENT"), 
                     layout_name=None, 
                     pagx_filename=None):
    """
    Exports a layout from an ArcGIS Pro project to a .pagx file.

    Parameters:
        aprx (project): project that contains the maps and the layers. The default argument is the current project.
        output_folder (str): Directory where the .pagx file will be saved.
        layout_name (str): Optional. Name of the layout to export. If None, the first layout is used.
        pagx_filename (str): Optional. Filename for the exported .pagx file. Defaults to layout name.
    
    Returns:
        str: Full path to the exported .pagx file.
    """
    import os
    try:
        if layout_name:
            layout = next((lyt for lyt in aprx.listLayouts() if lyt.name == layout_name), None)
            if not layout:
                raise ValueError(f"Layout named '{layout_name}' not found.")
        else:
            layout = aprx.listLayouts()[0]  # default to first layout

        # Determine filename
        if not pagx_filename:
            pagx_filename = f"{layout.name}.pagx"
        pagx_path = os.path.join(output_folder, pagx_filename)

        # Export the layout
        layout.exportToPAGX(pagx_path)

        print(f"Layout '{layout.name}' exported successfully to: {pagx_path}")
        return pagx_path

    except Exception as e:
        print(f"Error exporting layout: {e}")
        raise

In [27]:
pagx_path = save_layout_file(layout_name = 'Base Map 8.5x11P')
update_layout_datasources(pagx_path)

Layout 'Base Map 8.5x11P' exported successfully to: C:\Users\Spencer_C\Documents\UMP_Automation_Testing\testing\Base Map 8.5x11P.pagx


Labels for a natcom map are changed during a natcom maps creation. Change the labels back to default.

In [87]:
def reset_natcom_labels(aprx=arcpy.mp.ArcGISProject("CURRENT"),
                       local_path = r'C:\Users\Spencer_C\Documents\UMP_Automation_Testing\Layer_Files\Natural Communities.lyrx',
                       network_path = r'\\floridadep\data\DRP\GIS\LEGENDS\Layer_Files\Natural Communities.lyrx'):
    """
    resets the labels for natcoms to their default (no acreages)
    
    Parameters:
        
    Returns:
        
    """
    # Get the active map
    active_map = aprx.activeMap

    # Attempt to get the 'Natural Communities' layer
    for lyr in active_map.listLayers():
        try:
            if lyr.name == "Natural Communities (In Acres)" and not lyr.isGroupLayer:
                natcoms = lyr
        except Exception as e:
            print(f"Could not find \"Natural Communities (In Acres)\" layer: {e}")
            
    try: #try local copy
        lf = arcpy.mp.LayerFile(local_path)
        reference_lyr = lf.listLayers()[0]
    except:
        print('Local copy of the natcoms reference layer could not be found. The network copy was used instead.')
        lf = arcpy.mp.LayerFile(network_path)
        reference_lyr = lf.listLayers()[0]
    arcpy.management.ApplySymbologyFromLayer(natcoms, reference_lyr)

In [88]:
reset_natcom_labels()

Calculate the total area by natcom and update its label

In [ ]:
def update_natcom_labels(aprx=arcpy.mp.ArcGISProject("CURRENT")):
    """
    Updates the lables for natcoms to include the total area. 
    This function should be run after a definition query has been applied.
    
    Parameters:
        
    Returns:
        
    """
    # Get the active map
    active_map = aprx.activeMap

    # Attempt to get the 'Natural Communities' layer
    park_layer = None
    for lyr in active_map.listLayers():
        if lyr.name == "Natural Communities" and lyr.supports("SELECTION") and not lyr.isGroupLayer:
            park_layer = lyr
            break

Explore changing the symbology for active maps

In [100]:
# Get active map
aprx=arcpy.mp.ArcGISProject("CURRENT")
active_map = aprx.activeMap

# Get 'Natural Communities' layer
for lyr in active_map.listLayers():
    if lyr.name == "Natural Communities (In Acres)" and not lyr.isGroupLayer:
        natcoms = lyr

sym = natcoms.symbology
print(natcoms.symbology)

# Check if the layer's renderer is a UniqueValueRenderer
if natcoms.symbology.renderer.type == 'UniqueValueRenderer':
    renderer = natcoms.symbology.renderer

    # Iterate through unique values and modify their labels
    for unique_value in renderer.groups[0].items:
        # Example: Add a prefix to the label
        unique_value.label = "Prefix: " + unique_value.label
        #print(unique_value.label)

#natcoms.symbology = sym
print(natcoms.symbology)
    # Refresh the view to see the changes (if working in ArcMap)
    #arcpy.RefreshActiveView()
    # arcpy.RefreshTOC()


In [ ]:
apply_site_name_query("Lake Kissimmee State Park")
zoom_to_active_park()

In [ ]:
apply_site_name_query('Orman House Historic State Park')
zoom_to_active_park()

In [ ]:
remove_queries()

In [ ]:
save_layout_file(output_folder=r'C:\Users\Spencer_C\Documents\UMP_Automation_Testing\testing',
                layout_name='Base Map 8.5x11P')

In [ ]:
update_layer_datasources()
save_layout_file(output_folder = r'\\floridadep\data\DRP\GIS\PARKS\Orman House Historic State Park\Projects\UMP',
                 layout_name = 'Base Map 8.5x11P')
update_layer_datasources(old_path = r"\\floridadep\data\DRP\GIS\statewidecoverages\FloridaStateParksGIS.gdb", 
                         new_path = r"C:\Users\Spencer_C\Desktop\GDB Copy\FloridaStateParksGIS.gdb")

code below does not work because if there are more than ~3 maps open, the layers 'do not exist'

In [ ]:
'''
park_list = ['Orman House Historic State Park']
#park_list = ['Dr. Julian G. Bruce St. George Island State Park','Orman House Historic State Park','John Gorrie Museum State Park']
for park in park_list:
    for map_obj in aprx.listMaps():
        map_name = map_obj.name
        apply_site_name_query(map_name, park)
        #update_layer_datasources(map_name)
    #aprx.saveACopy(f'C:/Users/Spencer_C/Documents/UMP_Automation_Testing/testing/{park} UMP.aprx')
'''    